In [28]:
import pandas as pd
df = pd.DataFrame(columns=['ID', 'Pose ID'])
ids = ['ID' + str(i) for i in range(1, 6)]
pose_ids = ['Pose ' + str(i) for i in range(1, 11)]
# For each ID, add 10 rows to the dataframe with the corresponding Pose IDs
df_list = []
for i in ids:
    temp_df = pd.DataFrame({'ID': [i] * 10, 'Pose ID': pose_ids})
    df_list.append(temp_df)
df= pd.concat(df_list)
print(df)


    ID  Pose ID
0  ID1   Pose 1
1  ID1   Pose 2
2  ID1   Pose 3
3  ID1   Pose 4
4  ID1   Pose 5
5  ID1   Pose 6
6  ID1   Pose 7
7  ID1   Pose 8
8  ID1   Pose 9
9  ID1  Pose 10
0  ID2   Pose 1
1  ID2   Pose 2
2  ID2   Pose 3
3  ID2   Pose 4
4  ID2   Pose 5
5  ID2   Pose 6
6  ID2   Pose 7
7  ID2   Pose 8
8  ID2   Pose 9
9  ID2  Pose 10
0  ID3   Pose 1
1  ID3   Pose 2
2  ID3   Pose 3
3  ID3   Pose 4
4  ID3   Pose 5
5  ID3   Pose 6
6  ID3   Pose 7
7  ID3   Pose 8
8  ID3   Pose 9
9  ID3  Pose 10
0  ID4   Pose 1
1  ID4   Pose 2
2  ID4   Pose 3
3  ID4   Pose 4
4  ID4   Pose 5
5  ID4   Pose 6
6  ID4   Pose 7
7  ID4   Pose 8
8  ID4   Pose 9
9  ID4  Pose 10
0  ID5   Pose 1
1  ID5   Pose 2
2  ID5   Pose 3
3  ID5   Pose 4
4  ID5   Pose 5
5  ID5   Pose 6
6  ID5   Pose 7
7  ID5   Pose 8
8  ID5   Pose 9
9  ID5  Pose 10


In [27]:
from tqdm import tqdm
import itertools
import functools
import numpy as np
from IPython.display import display
def full_matrix_calculation(df):
    #Here I am using just string concatenation as an example calculation, in reality i am calling external functions
    def matrix_calculation(df, id_list):
        matrices = {}
        calc_dataframes = []
        for id in tqdm(id_list):
            df_name = df[df['ID']==id]
            df_name.index = range(len(df_name['Pose ID']))
            matrix = pd.DataFrame(0.0, index=[df_name['Pose ID']], columns=df_name['Pose ID'])
            for subset in itertools.combinations(df_name['Pose ID'], 2):
                result = subset[0]+subset[1]
                matrix.iloc[df_name[df_name['Pose ID']==subset[0]].index.values, df_name[df_name['Pose ID']==subset[1]].index.values] = result
                matrix.iloc[df_name[df_name['Pose ID']==subset[1]].index.values, df_name[df_name['Pose ID']==subset[0]].index.values] = result
            matrices[id] = matrix
        return matrices
    id_list = np.unique(np.array(df['ID']))
    calculated_dfs = matrix_calculation(df, id_list)
    return calculated_dfs

calculated_dfs = full_matrix_calculation(df)
display(calculated_dfs)

100%|██████████| 5/5 [00:00<00:00, 15.86it/s]


{'ID1': Pose ID         Pose 1         Pose 2         Pose 3         Pose 4  \
 Pose ID                                                               
 Pose 1             0.0   Pose 1Pose 2   Pose 1Pose 3   Pose 1Pose 4   
 Pose 2    Pose 1Pose 2            0.0   Pose 2Pose 3   Pose 2Pose 4   
 Pose 3    Pose 1Pose 3   Pose 2Pose 3            0.0   Pose 3Pose 4   
 Pose 4    Pose 1Pose 4   Pose 2Pose 4   Pose 3Pose 4            0.0   
 Pose 5    Pose 1Pose 5   Pose 2Pose 5   Pose 3Pose 5   Pose 4Pose 5   
 Pose 6    Pose 1Pose 6   Pose 2Pose 6   Pose 3Pose 6   Pose 4Pose 6   
 Pose 7    Pose 1Pose 7   Pose 2Pose 7   Pose 3Pose 7   Pose 4Pose 7   
 Pose 8    Pose 1Pose 8   Pose 2Pose 8   Pose 3Pose 8   Pose 4Pose 8   
 Pose 9    Pose 1Pose 9   Pose 2Pose 9   Pose 3Pose 9   Pose 4Pose 9   
 Pose 10  Pose 1Pose 10  Pose 2Pose 10  Pose 3Pose 10  Pose 4Pose 10   
 
 Pose ID         Pose 5         Pose 6         Pose 7         Pose 8  \
 Pose ID                                               

In [26]:
from tqdm import tqdm
import itertools
import functools
import numpy as np
from IPython.display import display
import multiprocessing
def function(tuple):
    tuple[1]
    return tuple[0]+tuple[1]
def full_matrix_calculation(df):
    #Here I am using just string concatenation as an example calculation, in reality i am calling external functions
    def matrix_calculation(df, id_list):
        matrices = {}
        calc_dataframes = []
        for id in tqdm(id_list):
            df_name = df[df['ID']==id]
            df_name.index = range(len(df_name['Pose ID']))
            matrix = pd.DataFrame(0.0, index=[df_name['Pose ID']], columns=df_name['Pose ID'])
            with multiprocessing.Pool() as p:
                try:
                    results = p.map(function, itertools.combinations(df_name['Pose ID'], 2))
                except KeyError:
                    print('Incorrect clustering method selected')
                    return
                results_list = list(zip(itertools.combinations(df_name['Pose ID'], 2), results))
                for subset, result in results_list:
                    matrix.iloc[df_name[df_name['Pose ID']==subset[0]].index.values, df_name[df_name['Pose ID']==subset[1]].index.values] = result
                    matrix.iloc[df_name[df_name['Pose ID']==subset[1]].index.values, df_name[df_name['Pose ID']==subset[0]].index.values] = result
                matrices[id] = matrix
            for subset in itertools.combinations(df_name['Pose ID'], 2):
                result = subset[0]+subset[1]
                matrix.iloc[df_name[df_name['Pose ID']==subset[0]].index.values, df_name[df_name['Pose ID']==subset[1]].index.values] = result
                matrix.iloc[df_name[df_name['Pose ID']==subset[1]].index.values, df_name[df_name['Pose ID']==subset[0]].index.values] = result
            matrices[id] = matrix
        return matrices
    id_list = np.unique(np.array(df['ID']))
    calculated_dfs = matrix_calculation(df, id_list)
    return calculated_dfs

calculated_dfs = full_matrix_calculation(df)
display(calculated_dfs)

100%|██████████| 5/5 [00:01<00:00,  4.30it/s]


{'ID1': Pose ID         Pose 1         Pose 2         Pose 3         Pose 4  \
 Pose ID                                                               
 Pose 1             0.0   Pose 1Pose 2   Pose 1Pose 3   Pose 1Pose 4   
 Pose 2    Pose 1Pose 2            0.0   Pose 2Pose 3   Pose 2Pose 4   
 Pose 3    Pose 1Pose 3   Pose 2Pose 3            0.0   Pose 3Pose 4   
 Pose 4    Pose 1Pose 4   Pose 2Pose 4   Pose 3Pose 4            0.0   
 Pose 5    Pose 1Pose 5   Pose 2Pose 5   Pose 3Pose 5   Pose 4Pose 5   
 Pose 6    Pose 1Pose 6   Pose 2Pose 6   Pose 3Pose 6   Pose 4Pose 6   
 Pose 7    Pose 1Pose 7   Pose 2Pose 7   Pose 3Pose 7   Pose 4Pose 7   
 Pose 8    Pose 1Pose 8   Pose 2Pose 8   Pose 3Pose 8   Pose 4Pose 8   
 Pose 9    Pose 1Pose 9   Pose 2Pose 9   Pose 3Pose 9   Pose 4Pose 9   
 Pose 10  Pose 1Pose 10  Pose 2Pose 10  Pose 3Pose 10  Pose 4Pose 10   
 
 Pose ID         Pose 5         Pose 6         Pose 7         Pose 8  \
 Pose ID                                               